<a href="https://colab.research.google.com/github/vanessajen/FlatLay-Generator/blob/main/FlatLay_Generator_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ControlNet Flatten V2 - Chuẩn Diffusers mới nhất

# --- Bước 0: Import thư viện cần thiết ---
!pip install diffusers transformers accelerate controlnet_aux --upgrade

# --- Bước 0.5: Set môi trường và tạo folder tự động ---
import os
os.environ["DIFFUSERS_FLAX_AVAILABLE"] = "0"

input_folder = "/content/drive/MyDrive/FlatLay_Input"
output_folder = "/content/drive/MyDrive/FlatLay_Demo_Output"

os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# --- Bước 1: Import thư viện Python ---
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import CannyDetector
from PIL import Image
import numpy as np

# --- Bước 2: Load model ControlNet ---
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# --- Bước 3: Khai báo Canny Detector ---
canny = CannyDetector()

# --- Bước 4: Hàm xử lý ảnh ---
def process_flatlay(input_image_path, prompt, output_path):
    image = Image.open(input_image_path)
    image = image.convert("RGB")
    image = image.resize((512, 512))

    # Tính toán canny map
    canny_image = canny(image)

    # Generate ảnh
    generated_image = pipe(
        prompt,
        num_inference_steps=30,
        guidance_scale=9.0,
        image=canny_image
    ).images[0]

    # Lưu ảnh
    generated_image.save(output_path)

# --- Bước 5: Quét toàn bộ ảnh và xử lý ---
prompt = "flat lay minimal pastel background, photography, highly detailed, white background"

has_input = False
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        has_input = True
        input_image_path = os.path.join(input_folder, filename)
        output_image_name = filename.split('.')[0] + "_flatlay.png"
        output_image_path = os.path.join(output_folder, output_image_name)

        process_flatlay(input_image_path, prompt, output_image_path)
        print(f"✅ Đã gen ảnh: {output_image_name}")

if not has_input:
    print(f"⚡️ Chưa có ảnh input. Vui lòng upload ảnh (.jpg/.png) vào: {input_folder}")
else:
    print("🌿✨ Done!")
